In [1]:
import pandas as pd
import numpy as np
import pylab 
import matplotlib.pyplot as plt
from collections import Counter
import math
import datetime as dt
import re
import itertools
import gc
from sklearn.metrics import mean_squared_error

In [2]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

2022-11-01 21:48:12.893952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
plt.rcParams['figure.figsize'] = [20, 5]

In [9]:
data =  pd.read_csv ('ETHUSDT.csv')
# data = pd.read_csv ('BTCUSDT.csv')
data.head()

,Open Time,Open Time.1,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,2017-08-17,2017-08-17 00:00:00.000000000,301.13,312.18,298.00,302.00,7030.71034,2017-08-17 23:59:59.999000064,2.154655e+06,4522,6224.58999,1.908705e+06,50579.872288
1,2017-08-18,2017-08-18 00:00:00.000000000,302.00,311.79,283.94,293.96,9537.84646,2017-08-18 23:59:59.999000064,2.858947e+06,5658,7452.43542,2.240813e+06,55329.574324
2,2017-08-19,2017-08-19 00:00:00.000000000,293.31,299.90,278.00,290.91,2146.19773,2017-08-19 23:59:59.999000064,6.200226e+05,1795,1537.97550,4.447136e+05,58005.171046
3,2017-08-20,2017-08-20 00:00:00.000000000,289.41,300.53,282.85,299.10,2510.13871,2017-08-20 23:59:59.999000064,7.428479e+05,2038,2186.68739,6.475061e+05,60299.066154
4,2017-08-21,2017-08-21 00:00:00.000000000,299.10,346.52,294.60,323.29,5219.44542,2017-08-21 23:59:59.999000064,1.689472e+06,3925,4004.19967,1.297977e+06,66203.840222


In [10]:
data['Change_close'] = data['Close'].pct_change()
data['Change_close'] = data['Change_close'].fillna(0)
data['Date'] = data['Open Time']

In [11]:
data.head()

,Open Time,Open Time.1,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore,Change_close,Date
0,2017-08-17,2017-08-17 00:00:00.000000000,301.13,312.18,298.00,302.00,7030.71034,2017-08-17 23:59:59.999000064,2.154655e+06,4522,6224.58999,1.908705e+06,50579.872288,0.000000,2017-08-17
1,2017-08-18,2017-08-18 00:00:00.000000000,302.00,311.79,283.94,293.96,9537.84646,2017-08-18 23:59:59.999000064,2.858947e+06,5658,7452.43542,2.240813e+06,55329.574324,-0.026623,2017-08-18
2,2017-08-19,2017-08-19 00:00:00.000000000,293.31,299.90,278.00,290.91,2146.19773,2017-08-19 23:59:59.999000064,6.200226e+05,1795,1537.97550,4.447136e+05,58005.171046,-0.010376,2017-08-19
3,2017-08-20,2017-08-20 00:00:00.000000000,289.41,300.53,282.85,299.10,2510.13871,2017-08-20 23:59:59.999000064,7.428479e+05,2038,2186.68739,6.475061e+05,60299.066154,0.028153,2017-08-20
4,2017-08-21,2017-08-21 00:00:00.000000000,299.10,346.52,294.60,323.29,5219.44542,2017-08-21 23:59:59.999000064,1.689472e+06,3925,4004.19967,1.297977e+06,66203.840222,0.080876,2017-08-21


In [22]:
def train_test(X):
    dataset = X[['Date','Close','Change_close']]
    train_size = int(len(dataset) * 0.95)
    train, test = dataset[0:train_size], dataset[train_size:] 
    return train,test

In [23]:
# Accuracy of the model
def performance(y_true, y_pred): 
    mse = ((y_pred - y_true) ** 2).mean()
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return( print('MSE of predictions is {}'.format(round(mse, 2))+
                  '\nRMSE of predictions is {}'.format(round(np.sqrt(mse), 2))+
                  '\nMAPE of predictions is {}'.format(round(mape, 2))))

In [ ]:
# Reference Assignment 5 Beijing Handout
def describe_full(df):
    #pd.options.display.float_format = '{:.2f}'.format
    dtypes_description=pd.DataFrame(dict(df.dtypes),["dtypes"])
    na_description = pd.DataFrame(dict(df.isna().sum()),["NA-s"])
    na_percent = ((pd.DataFrame(dict(df.isna().sum()),["NA%"])/len(df))*100).round(decimals=2)
    description = df.describe(include='all')
    full_description = dtypes_description.append(na_description).append(na_percent).append(description).replace(np.nan, '', regex=True)

    mask = full_description.loc["freq",:]==1
    full_description.at[["top"],mask.index[mask]]=""
    #TODO: scientific notation - could be nicer
    
    return full_description